# 魔法方法
## Magic Method
- 外观：前后各有两个下划线的方法，如：```__add__```
- 用途：自定义类中定义了魔法方法，就可以享受到内置类型的*待遇*；
  - 使用运算符表示运算：```+ - * / // < > == in += *=```
  - 使用下标索引分量：```[i] [start:end:step] [key]```
  - 使用内置函数：```sort sum```
- 官方名称：特殊方法（Special Method）
  - https://docs.python.org/zh-cn/3/reference/datamodel.html#special-method-names

## 我们来定义一个类“魔法存钱罐” ```MagicPiggyBank```
- 每个存钱罐可以装1，2，5分钱的硬币若干；
  - ```toms = MagicPiggyBank(owner="Tom", total=11)```
  - 存钱罐的魔法使得装进去的硬币可以自动合并到**最少**数量；
- 可以打印出存钱罐内容；
  - ```print(toms)```
- 可以查询每种硬币的数量；
  - ```print(toms["one"], toms["two"], toms["five"])```
  - ```print(toms[1], toms[2], toms[5])```
- 可以判断罐子里是否有某种硬币；
  - ```"one" in toms```
  - ```1 in toms```
- 可以从罐子里取钱，也可以存钱进罐子，甚至可以魔法倍增(各种硬币数量倍增)；
  - ```toms += 8```
  - ```toms -= 7```
  - ```toms *= 2```
- 两个罐子可以合并；
  - ```mikes = toms + jerrys```
- 若干罐子也可以合并；
  - ```alls = sum([toms, jerrys, mikes])```
- 若干罐子可以根据**硬币数量**从少到多排序
  - ```sorted([toms, jerrys, mikes])```

![](piggybank.jpg)

## 类定义代码

In [2]:
# 魔法存钱罐 #
from typing import Self

class MagicPiggyBank:
    def __init__(self, owner="Anony", total=0) -> None:
        self.owner = owner
        self.total = total
        self.merge()

    def merge(self) -> None:  # 自动合并为最少数量
        t = self.total
        five, t = divmod(t, 5)
        two, t = divmod(t, 2)
        self.coins = (t, two, five)

    def __str__(self) -> str:  # 字符串形式输出
        return (
            f"[{sum(self.coins)}]{self.owner}'s: (1)*{self.coins[0]} "
            f"(2)*{self.coins[1]} (5)*{self.coins[2]} total={self.total}"
        )

    __repr__ = __str__

    def __len__(self) -> int:  # 硬币数量
        return sum(self.coins)

    def __lt__(self, other) -> bool:  # 比较
        return len(self) < len(other)

    def __getitem__(self, coin) -> int:  # 获取指定硬币数量
        match coin:
            case 1 | "one":
                return self.coins[0]
            case 2 | "two":
                return self.coins[1]
            case 5 | "five":
                return self.coins[2]
            case _:
                return 0

    def __contains__(self, coin) -> bool:  # 判断指定硬币是否存在
        return self[coin] > 0

    def __add__(self, other: Self) -> Self:  # 存钱罐合并
        result = MagicPiggyBank(owner=self.owner, total=self.total + other.total)
        return result

    def __iadd__(self, money: int) -> Self:  # 存钱罐加钱
        if money < 0:
            raise ValueError("Cannot thrink.")
        else:
            self.total += money
            self.merge()
        return self

    def __isub__(self, money: int) -> Self:  # 存钱罐取钱，无法透支
        if self.total < money:
            raise ValueError("Insufficient balance.")
        else:
            self.total -= money
            self.merge()
        return self

    def __imul__(self, times: int) -> Self:  # 存钱罐倍增
        if times < 0:
            raise ValueError("Invalid times.")
        else:
            self.total *= times
            self.merge()
        return self

    def __abs__(self) -> int:  # 总金额
        return self.total

    def __radd__(self, start: int) -> Self:  # 用于sum的0+MagicPiggyBank
        self.total += start
        self.merge()
        return self


## 魔法方法的应用


In [3]:
# Tom的存钱罐
toms = MagicPiggyBank(owner="Tom", total=11)
print(toms)
print("硬币数量：", len(toms), "总金额：", abs(toms))
print("1分多少枚：", toms[1], "5分多少枚：", toms["five"])
print("罐中有2分硬币吗？", "two" in toms)


[3]Tom's: (1)*1 (2)*0 (5)*2 total=11
硬币数量： 3 总金额： 11
1分多少枚： 1 5分多少枚： 2
罐中有2分硬币吗？ False


In [4]:
# 加12分到罐子里
toms += 12
print(toms)


[6]Tom's: (1)*1 (2)*1 (5)*4 total=23


In [5]:
# 减4分
toms -= 4
print(toms)


[5]Tom's: (1)*0 (2)*2 (5)*3 total=19


In [6]:
# 倍增2倍
toms *= 2
print(toms)


[9]Tom's: (1)*1 (2)*1 (5)*7 total=38


In [7]:
# 合并运算
jerrys = MagicPiggyBank(owner="Jerry", total=2)
mikes = toms + jerrys
mikes.owner = "Mike"
print(jerrys)
print(mikes)


[1]Jerry's: (1)*0 (2)*1 (5)*0 total=2
[8]Mike's: (1)*0 (2)*0 (5)*8 total=40


In [8]:
# 合成大钱罐
alls = sum([toms, jerrys, mikes])
alls.owner = "All"
print(alls)


[16]All's: (1)*0 (2)*0 (5)*16 total=80


In [9]:
# 按照硬币数量排序
print(sorted([toms, jerrys, mikes]))


[[1]Jerry's: (1)*0 (2)*1 (5)*0 total=2, [8]Mike's: (1)*0 (2)*0 (5)*8 total=40, [9]Tom's: (1)*1 (2)*1 (5)*7 total=38]
